<a href="https://colab.research.google.com/github/anna1802002/Healthcare-Price-Prediction/blob/main/Healthcare_price_prediction_using_provider_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Load datasets with specified encoding 'latin1'
enrollment_df = pd.read_csv('/content/drive/MyDrive/Medicare Fee-For-Service  Public Provider Enrollment/2024-Q2/PPEF_Enrollment_Extract_2024.07.01.csv', encoding='latin1')
location_df = pd.read_csv('/content/drive/MyDrive/Medicare Fee-For-Service  Public Provider Enrollment/2024-Q2/PPEF_Practice_Location_Extract_2024.07.01.csv', encoding='latin1')
reassignment_df = pd.read_csv('/content/drive/MyDrive/Medicare Fee-For-Service  Public Provider Enrollment/2024-Q2/PPEF_Reassignment_Extract_2024.07.01.csv', encoding='latin1')
specialty_df = pd.read_csv('/content/drive/MyDrive/Medicare Fee-For-Service  Public Provider Enrollment/2024-Q2/PPEF_Secondary_Specialty_Extract_2024.07.01.csv', encoding='latin1')




<ipython-input-13-7c9a276a620c>:6: DtypeWarning: Columns (6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  enrollment_df = pd.read_csv('/content/drive/MyDrive/Medicare Fee-For-Service  Public Provider Enrollment/2024-Q2/PPEF_Enrollment_Extract_2024.07.01.csv', encoding='latin1')


In [14]:
# Display basic information to understand the structure
print("Enrollment Data Info")
print(enrollment_df.info())
print(enrollment_df.head())

print("\nPractice Location Data Info")
print(location_df.info())
print(location_df.head())

print("\nReassignment Data Info")
print(reassignment_df.info())
print(reassignment_df.head())

print("\nSpecialty Data Info")
print(specialty_df.info())
print(specialty_df.head())


Enrollment Data Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2844889 entries, 0 to 2844888
Data columns (total 11 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   NPI                 int64 
 1   PECOS_ASCT_CNTL_ID  int64 
 2   ENRLMT_ID           object
 3   PROVIDER_TYPE_CD    object
 4   PROVIDER_TYPE_DESC  object
 5   STATE_CD            object
 6   FIRST_NAME          object
 7   MDL_NAME            object
 8   LAST_NAME           object
 9   ORG_NAME            object
 10  GNDR_SW             object
dtypes: int64(2), object(9)
memory usage: 238.8+ MB
None
          NPI  PECOS_ASCT_CNTL_ID        ENRLMT_ID PROVIDER_TYPE_CD  \
0  1003879883          8022920719  I20031103000001            14-16   
1  1003976986          7113839812  I20031103000005            14-68   
2  1407802119          8022920727  I20031103000007            14-93   
3  1831165075          5193637890  I20031103000013            14-16   
4  1851357214          2466364161  I20

In [15]:
# Check for missing values in each dataset
print("\nMissing values in Enrollment Data")
print(enrollment_df.isnull().sum())

print("\nMissing values in Practice Location Data")
print(location_df.isnull().sum())

print("\nMissing values in Reassignment Data")
print(reassignment_df.isnull().sum())

print("\nMissing values in Specialty Data")
print(specialty_df.isnull().sum())

# Handling missing values by forward filling
enrollment_df.fillna(method='ffill', inplace=True)
location_df.fillna(method='ffill', inplace=True)
reassignment_df.fillna(method='ffill', inplace=True)
specialty_df.fillna(method='ffill', inplace=True)

# Additional check for any remaining missing values
print("\nMissing values after forward filling:")
print("Enrollment Data:\n", enrollment_df.isnull().sum())
print("Practice Location Data:\n", location_df.isnull().sum())
print("Reassignment Data:\n", reassignment_df.isnull().sum())
print("Specialty Data:\n", specialty_df.isnull().sum())



Missing values in Enrollment Data
NPI                         0
PECOS_ASCT_CNTL_ID          0
ENRLMT_ID                   0
PROVIDER_TYPE_CD            0
PROVIDER_TYPE_DESC          0
STATE_CD                    0
FIRST_NAME             443867
MDL_NAME              1269526
LAST_NAME              443907
ORG_NAME              2401047
GNDR_SW                771861
dtype: int64

Missing values in Practice Location Data
ENRLMT_ID    0
CITY_NAME    0
STATE_CD     0
ZIP_CD       0
dtype: int64

Missing values in Reassignment Data
REASGN_BNFT_ENRLMT_ID    0
RCV_BNFT_ENRLMT_ID       0
dtype: int64

Missing values in Specialty Data
ENRLMT_ID             0
PROVIDER_TYPE_CD      0
PROVIDER_TYPE_DESC    0
dtype: int64

Missing values after forward filling:
Enrollment Data:
 NPI                         0
PECOS_ASCT_CNTL_ID          0
ENRLMT_ID                   0
PROVIDER_TYPE_CD            0
PROVIDER_TYPE_DESC          0
STATE_CD                    0
FIRST_NAME                  0
MDL_NAME         

In [16]:
print("Enrollment DataFrame columns:", enrollment_df.columns)
print("Location DataFrame columns:", location_df.columns)
print("Reassignment DataFrame columns:", reassignment_df.columns)
print("Specialty DataFrame columns:", specialty_df.columns)


Enrollment DataFrame columns: Index(['NPI', 'PECOS_ASCT_CNTL_ID', 'ENRLMT_ID', 'PROVIDER_TYPE_CD',
       'PROVIDER_TYPE_DESC', 'STATE_CD', 'FIRST_NAME', 'MDL_NAME', 'LAST_NAME',
       'ORG_NAME', 'GNDR_SW'],
      dtype='object')
Location DataFrame columns: Index(['ENRLMT_ID', 'CITY_NAME', 'STATE_CD', 'ZIP_CD'], dtype='object')
Reassignment DataFrame columns: Index(['REASGN_BNFT_ENRLMT_ID', 'RCV_BNFT_ENRLMT_ID'], dtype='object')
Specialty DataFrame columns: Index(['ENRLMT_ID', 'PROVIDER_TYPE_CD', 'PROVIDER_TYPE_DESC'], dtype='object')


In [17]:
# Merge datasets on 'ENRLMT_ID'
merged_df = enrollment_df.merge(location_df, on='ENRLMT_ID', how='left')
merged_df = merged_df.merge(reassignment_df, left_on='ENRLMT_ID', right_on='REASGN_BNFT_ENRLMT_ID', how='left')
merged_df = merged_df.merge(specialty_df, on='ENRLMT_ID', how='left')

# Use 'PROVIDER_TYPE_CD_y' for counting specialties
merged_df['Specialty_Count'] = merged_df.groupby('ENRLMT_ID')['PROVIDER_TYPE_CD_y'].transform('count')

# Use 'ZIP_CD' for counting locations
merged_df['Location_Count'] = merged_df.groupby('ENRLMT_ID')['ZIP_CD'].transform('count')

# Drop duplicates if any
merged_df = merged_df.drop_duplicates(subset='ENRLMT_ID')

# Display the first few rows of the merged dataframe
print(merged_df.head())


          NPI  PECOS_ASCT_CNTL_ID        ENRLMT_ID PROVIDER_TYPE_CD_x  \
0  1003879883          8022920719  I20031103000001              14-16   
2  1003976986          7113839812  I20031103000005              14-68   
3  1407802119          8022920727  I20031103000007              14-93   
5  1831165075          5193637890  I20031103000013              14-16   
6  1851357214          2466364161  I20031103000014              14-30   

                   PROVIDER_TYPE_DESC_x STATE_CD_x   FIRST_NAME MDL_NAME  \
0  PRACTITIONER - OBSTETRICS/GYNECOLOGY         PR      ANTONIO      NaN   
2  PRACTITIONER - CLINICAL PSYCHOLOGIST         PA  CHRISTOPHER        J   
3     PRACTITIONER - EMERGENCY MEDICINE         PA      KADISHA        B   
5  PRACTITIONER - OBSTETRICS/GYNECOLOGY         PR        JORGE        A   
6   PRACTITIONER - DIAGNOSTIC RADIOLOGY         KY       RHONDA        G   

           LAST_NAME ORG_NAME GNDR_SW      CITY_NAME STATE_CD_y       ZIP_CD  \
0  ALVAREZ RODRIGUEZ    

In [18]:
print("Columns in merged_df:", merged_df.columns)


Columns in merged_df: Index(['NPI', 'PECOS_ASCT_CNTL_ID', 'ENRLMT_ID', 'PROVIDER_TYPE_CD_x',
       'PROVIDER_TYPE_DESC_x', 'STATE_CD_x', 'FIRST_NAME', 'MDL_NAME',
       'LAST_NAME', 'ORG_NAME', 'GNDR_SW', 'CITY_NAME', 'STATE_CD_y', 'ZIP_CD',
       'REASGN_BNFT_ENRLMT_ID', 'RCV_BNFT_ENRLMT_ID', 'PROVIDER_TYPE_CD_y',
       'PROVIDER_TYPE_DESC_y', 'Specialty_Count', 'Location_Count'],
      dtype='object')


Model Development

In [19]:
from sklearn.preprocessing import StandardScaler

# Scale the numerical features
scaler = StandardScaler()

# Apply scaling to 'Specialty_Count' and 'Location_Count'
merged_df[['Specialty_Count_Scaled', 'Location_Count_Scaled']] = scaler.fit_transform(merged_df[['Specialty_Count', 'Location_Count']])

# Check scaled features
print(merged_df[['Specialty_Count_Scaled', 'Location_Count_Scaled']].head())


   Specialty_Count_Scaled  Location_Count_Scaled
0               -0.247981              -0.072841
2               -0.247981               0.112867
3               -0.247981              -0.072841
5               -0.247981               0.112867
6               -0.247981              -0.072841


In [22]:
import numpy as np
# Create interaction features between Specialty_Count and Location_Count
merged_df['Specialty_Location_Interaction'] = merged_df['Specialty_Count'] * merged_df['Location_Count']

# Interaction between Specialty_Count and the logarithmic version of Location_Count
merged_df['Specialty_LogLocation_Interaction'] = merged_df['Specialty_Count'] * np.log1p(merged_df['Location_Count'])

# Check interaction features
print(merged_df[['Specialty_Location_Interaction', 'Specialty_LogLocation_Interaction']].head())


   Specialty_Location_Interaction  Specialty_LogLocation_Interaction
0                               0                                0.0
2                               0                                0.0
3                               0                                0.0
5                               0                                0.0
6                               0                                0.0


In [24]:
# Define a function to group provider types into broader categories
def group_provider_types(provider_type):
    if provider_type in ['Specialty1', 'Specialty2']:  # Replace with actual categories
        return 'Group1'
    elif provider_type in ['Specialty3', 'Specialty4']:  # Replace with actual categories
        return 'Group2'
    else:
        return 'Other'

# Apply grouping to the 'PROVIDER_TYPE_CD_y' column to create 'Provider_Type_Grouped'
merged_df['Provider_Type_Grouped'] = merged_df['PROVIDER_TYPE_CD_y'].apply(group_provider_types)

# Check if the new column is created
print(merged_df[['PROVIDER_TYPE_CD_y', 'Provider_Type_Grouped']].head())


  PROVIDER_TYPE_CD_y Provider_Type_Grouped
0                NaN                 Other
2                NaN                 Other
3                NaN                 Other
5                NaN                 Other
6                NaN                 Other


In [25]:
# One-hot encoding the 'Provider_Type_Grouped' column
merged_df = pd.get_dummies(merged_df, columns=['Provider_Type_Grouped'], drop_first=True)

# Check one-hot encoded columns
print(merged_df.filter(like='Provider_Type_Grouped').head())


Empty DataFrame
Columns: []
Index: [0, 2, 3, 5, 6]
